In [1]:
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline


import time as t

In [2]:
import pandas as pd
import sys
import pybullet as p
from env_pybullet_gen3 import env_pybullet_kin_gen3



In [3]:
#To improve the velocity, run it on the GPU


#Create a experiment env
env = env_pybullet_kin_gen3(no_zeros = True,Excels_path_Okay_tcp_list = ["Tcp_Trajectori_19.xlsx","Tcp_Trajectori_39.xlsx","Tcp_Trajectori_59.xlsx"],Actions_offsets_Numpy_files = ["acs_19_tcp.npy","acs_39_tcp.npy","acs_59_tcp.npy"],time_step=0.02,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = False

#Initially parameters of the urdf

#Make maxvels closer to the reality
#both have to be modified
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
print(env.original_parameters_df["max_vel"])
print(env.original_parameters_df)


print('observation space:', env.observation_space) #states, There is only 1 state constant
env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
print('action space:', env.action_space) #parameters, number of parameters choose to tune, continuous
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous




hola
../Simulation_Pybullet/models/urdf/JACO3_URDF_V11.urdf
Robot launched
hola
(7, 12)
(7, 17)
0    168
1    151
2    128
3     94
4    210
5     48
6    189
Name: max_vel, dtype: int64
       mass damping       Ixx       Iyy       Izz   kp   ki   kd  max_vel  \
0  1.377353       0  0.004801  0.004755  0.002283  0.1  0.0  0.0      168   
1  1.163667       0  0.008419  0.001920  0.008361  0.1  0.0  0.0      151   
2  1.163660       0  0.007545  0.007487  0.001921  0.1  0.0  0.0      128   
3  0.930287       0  0.006410  0.001380  0.006518  0.1  0.0  0.0       94   
4  0.678106       0  0.001680  0.001506  0.000826  0.1  0.0  0.0      210   
5  0.678106       0  0.001938  0.000827  0.001763  0.1  0.0  0.0       48   
6  0.500657       0  0.000775  0.000585  0.000975  0.1  0.0  0.0      189   

   force_x_one  
0            1  
1            1  
2            1  
3            1  
4            1  
5            1  
6            1  
observation space: 1
mass okey
max_vel okey
kp okey
ki okey


In [4]:
#Cross Entrophy Method, to choose the weights

# In my case where only 1 action,and that it's apply the parameters do another step doesn't change anything due to the state doesn't vary
# For this reason max_t and gama doesn't make sense, so I set them to max_t = 1 and gamma to 0
def cem_no_net(n_iterations=700, max_t=1, gamma=0.0, print_every=50, pop_size=env.action_space, elite_frac=10/(env.action_space*2), sigma=0.3,sigma_reduction_every_print = 0.65, per_one = True ):
    """PyTorch implementation of the cross-entropy method.
        
    Params
    ======
        n_iterations (int): maximum number of training iterations
        max_t (int): maximum number of timesteps per episode
        gamma (float): discount rate
        print_every (int): how often to print average score (over last 100 episodes)
        pop_size (int): size of population at each iteration
        elite_frac (float): percentage of top performers to use in update
        sigma (float): standard deviation of additive noise
        per_one (boolean): to determine if the output is in per one or not
    """
    #Numbers of elements that you keep as the better ones
    n_elite=int(pop_size*elite_frac)
    
    #scores doble end queee , from iterations size * 0.1
    scores_deque = deque(maxlen=int(n_iterations*0.1))
    #intial scores empty
    scores = []
    #Save actions to see how they evolve
    best_actions = []
    #Select a seed to make the results the same every test, not depending on the seed
    np.random.seed(0)
    #Initial best weights, are from 0 to 1, it's good to be small the weights, but they should be different from 0.
    # small to avoid overfiting , different from 0 to update them
    
    if (per_one == True):
        best_weight = sigma*np.random.randn(env.action_space)
        original_action = np.array(env.action_original())
    else:
        best_weight = np.add(sigma*np.random.randn(env.action_space),env.action_original())

    #Each iteration, modify  + (from 0 to 1) the best weight randomly
    #Computes the reward with these weights
    #Sort the reward to get the best ones
    # Save the best weights
    # the Best weight it's the mean of the best one
    #compute the main reward of the main best rewards ones
    #this it's show to evalute how good its
    
    for i_iteration in range(1, n_iterations+1):
        
        #Generate new population weights, as a mutation of the best weight to test them
        weights_pop = [best_weight + (sigma*np.random.randn(env.action_space)) for i in range(pop_size)]
        
        #Compute the parameters and obtain the rewards for each of them
        #print("iteration "+str(i_iteration))
        if (per_one == True):
            rewards=[]
            for weights in weights_pop:
                #print("New weights")
                #print(weights)
                #t.sleep(1000)
                action=np.add(np.multiply(weights,original_action),original_action)
                rewards.append( env.step_tcp_rishabh_actions(action) )
            rewards = np.array(rewards)
        else:
            rewards=[]
            for weights in weights_pop:
                rewards.append(env.step_tcp_rishabh_actions(weights) )
            rewards = np.array(rewards)
        print("rewards" + str(i_iteration))
        print(rewards)
        #print("\n")
        
        #Sort the rewards to obtain the best ones
        elite_idxs = rewards.argsort()[-n_elite:]
        elite_weights = [weights_pop[i] for i in elite_idxs]
        
        #Set the best weight as the mean of the best ones 
       
        best_weight = np.array(elite_weights).mean(axis=0)
        
        #Get the reward with this new weight
        
        if (per_one == True):
            action = np.add(np.multiply(best_weight,original_action),original_action)
            #print("action vel")
            #print(action[8])
            #print("action kp")
            #print(action[15])
            #time.sleep(1000)
            best_actions.append(action)
            reward = env.step_tcp_rishabh_actions(action)
            print("reward")
            print(reward)
            print("\n")
        else:
            best_actions.append(best_weight)
            reward = env.step_tcp_rishabh_actions(best_weight)
        scores_deque.append(reward)
        scores.append(reward)
        
        #save the check point
        env.save_parameters("./Parameters_train_tcp_euc_rishabh.xlsx")
        
        if i_iteration % print_every == 0:
            sigma = sigma * sigma_reduction_every_print
            print('Episode {}\tAverage Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))

        if np.mean(scores_deque)>=0.0:
            print('\nEnvironment solved in {:d} iterations!\tAverage Score: {:.2f}'.format(i_iteration-n_iterations*0.1, np.mean(scores_deque)))
            break
    return scores,best_actions


In [5]:
#Execute the cross entrophy method with default Values
#scores = cem()


#To don't ask the GPU as much reduce the pop_size, it's the amount of elemts try
scores,best_actions = cem_no_net()
# 
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
    

[]
Actions_provided 100
Actions_provided 100
Actions_provided 100
self.df_Okay        pos 0     pos 1     pos 2    quat 0    quat 1    quat 2    quat 3
0   0.213630 -0.137126  0.243382  0.005138  0.710665  0.703512  0.000061
1   0.231112 -0.144910  0.262846  0.003709  0.709538  0.704657  0.000207
2   0.248387 -0.144779  0.282388  0.001961  0.709188  0.705016  0.000193
3   0.264775 -0.143841  0.301897  0.000769  0.708927  0.705282 -0.000246
4   0.279280 -0.147114  0.316552  0.000660  0.708314  0.705897 -0.000496
..       ...       ...       ...       ...       ...       ...       ...
95  0.343016  0.191357  0.248591  0.000419  0.707005  0.707208  0.000078
96  0.344213  0.192157  0.248261  0.000321  0.707014  0.707200  0.000047
97  0.344028  0.192749  0.248618  0.000176  0.707042  0.707171  0.000039
98  0.342666  0.191990  0.249097  0.000114  0.707107  0.707107  0.000025
99  0.341531  0.191725  0.249089  0.000031  0.707109  0.707105  0.000006

[100 rows x 7 columns]
tcp_pos [[ 0.21361004

ValueError: operands could not be broadcast together with shapes (599,) (99,) 

In [ ]:

# plot the last scores zoom
fig = plt.figure()
zoom= 300
ax = fig.add_subplot(111)
plt.plot(np.arange(1, zoom+1), scores[-zoom:])
plt.ylabel('Score zoom')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# plot actions
best_actions_np = np.array(best_actions)
for joint in range(7):
    for i in range(len(env.parameters_to_modify)) :
        parameter = env.parameters_to_modify[i]
        figures = plt.figure()
        ax = fig.add_subplot(111)
        plt.plot(np.arange(1, best_actions_np.shape[0]+1), best_actions_np[:,joint+i*7])
        plt.ylabel(parameter+" Joint"+str(joint))
        plt.xlabel('Episode #')
        plt.show()

In [ ]:
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./positions_from_joints_19.xlsx",time_step=0.05,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = True

#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz","damping"])
env.robot.visual_inspection = False
env.modified_parameters_df = env.create_df_from_Excel("./Parameters_train_tcp_euc_rishabh.xlsx")


t.sleep(0.02)
action = env.action_modified()
action = np.array(action)
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous
print("trained",action)
reward = env.step_tcp_rishabh(action)
print("reward")
print(reward)



In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Train_parameters_result_tcp_euc_rishabh.xlsx")

In [ ]:
env.original_parameters_df

In [ ]:
#Original ERROR
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excels_path_Okay_tcp_list = ["Tcp_Trajectori_19.xlsx","Tcp_Trajectori_39.xlsx","Tcp_Trajectori_59.xlsx"],\
                            Actions_offsets_Numpy_files = ["acs_19_tcp.npy","acs_39_tcp.npy","acs_59_tcp.npy"],time_step=0.02,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
env.robot.visual_inspection = True
env.modified_parameters_df = env.original_parameters_df


t.sleep(10)
action = env.action_modified()
action = np.array(action)
print(action)
reward = env.step_tcp_rishabh_actions(action)

print("reward")
print(reward)

hola
../Simulation_Pybullet/models/urdf/JACO3_URDF_V11.urdf
Robot launched
hola
(7, 12)
(7, 17)
mass okey
max_vel okey
kp okey
ki okey
kd okey
force_x_one okey
Ixx okey
Iyy okey
Izz okey
[1.37735300e+00 1.16366700e+00 1.16366000e+00 9.30287000e-01
 6.78106000e-01 6.78106000e-01 5.00657000e-01 1.68000000e+02
 1.51000000e+02 1.28000000e+02 9.40000000e+01 2.10000000e+02
 4.80000000e+01 1.89000000e+02 1.00000000e-01 1.00000000e-01
 1.00000000e-01 1.00000000e-01 1.00000000e-01 1.00000000e-01
 1.00000000e-01 5.00000000e-02 5.00000000e-02 5.00000000e-02
 5.00000000e-02 5.00000000e-02 5.00000000e-02 5.00000000e-02
 5.00000000e-02 5.00000000e-02 5.00000000e-02 5.00000000e-02
 5.00000000e-02 5.00000000e-02 5.00000000e-02 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 4.80078221e-03 8.41872412e-03
 7.54451620e-03 6.40969196e-03 1.67978468e-03 1.93759353e-03
 7.75038580e-04 4.75519127e-03 1.92020573e-03 7.48660506e-03
 1.38041303e-03 1.50

Try  39  error:  0.05009964835160871
Try  40  error:  0.05009964835160871
Try  41  error:  0.05009964835160871
Try  42  error:  0.05009964835160871
Try  43  error:  0.05009964835160871
Try  44  error:  0.05009964835160871
Try  45  error:  0.05009964835160871
Try  46  error:  0.05009964835160871
Try  47  error:  0.05009964835160871
Try  48  error:  0.05009964835160871
Try  49  error:  0.05009964835160871
Try  50  error:  0.05009964835160871
Try  51  error:  0.05009964835160871
Try  52  error:  0.05009964835160871
Try  53  error:  0.05009964835160871
Try  54  error:  0.05009964835160871
Try  55  error:  0.05009964835160871
Try  56  error:  0.05009964835160871
Try  57  error:  0.05009964835160871
Try  58  error:  0.05009964835160871
Try  59  error:  0.05009964835160871
Try  60  error:  0.05009964835160871
Try  61  error:  0.05009964835160871
Try  62  error:  0.05009964835160871
Try  63  error:  0.05009964835160871
Try  64  error:  0.05009964835160871
Try  65  error:  0.05009964835160871
T

Try  258  error:  0.05009964835160871
Try  259  error:  0.05009964835160871
Try  260  error:  0.05009964835160871
Try  261  error:  0.05009964835160871
Try  262  error:  0.05009964835160871
Try  263  error:  0.05009964835160871
Try  264  error:  0.05009964835160871
Try  265  error:  0.05009964835160871
Try  266  error:  0.05009964835160871
Try  267  error:  0.05009964835160871
Try  268  error:  0.05009964835160871
Try  269  error:  0.05009964835160871
Try  270  error:  0.05009964835160871
Try  271  error:  0.05009964835160871
Try  272  error:  0.05009964835160871
Try  273  error:  0.05009964835160871
Try  274  error:  0.05009964835160871
Try  275  error:  0.05009964835160871
Try  276  error:  0.05009964835160871
Try  277  error:  0.05009964835160871
Try  278  error:  0.05009964835160871
Try  279  error:  0.05009964835160871
Try  280  error:  0.05009964835160871
Try  281  error:  0.05009964835160871
Try  282  error:  0.05009964835160871
Try  283  error:  0.05009964835160871
Try  284  er

Try  474  error:  0.05009964835160871
Try  475  error:  0.05009964835160871
Try  476  error:  0.05009964835160871
Try  477  error:  0.05009964835160871
Try  478  error:  0.05009964835160871
Try  479  error:  0.05009964835160871
Try  480  error:  0.05009964835160871
Try  481  error:  0.05009964835160871
Try  482  error:  0.05009964835160871
Try  483  error:  0.05009964835160871
Try  484  error:  0.05009964835160871
Try  485  error:  0.05009964835160871
Try  486  error:  0.05009964835160871
Try  487  error:  0.05009964835160871
Try  488  error:  0.05009964835160871
Try  489  error:  0.05009964835160871
Try  490  error:  0.05009964835160871
Try  491  error:  0.05009964835160871
Try  492  error:  0.05009964835160871
Try  493  error:  0.05009964835160871
Try  494  error:  0.05009964835160871
Try  495  error:  0.05009964835160871
Try  496  error:  0.05009964835160871
Try  497  error:  0.05009964835160871
Try  498  error:  0.05009964835160871
Try  499  error:  0.05009964835160871
Try  500  er

Try  691  error:  0.05009964835160871
Try  692  error:  0.05009964835160871
Try  693  error:  0.05009964835160871
Try  694  error:  0.05009964835160871
Try  695  error:  0.05009964835160871
Try  696  error:  0.05009964835160871
Try  697  error:  0.05009964835160871
Try  698  error:  0.05009964835160871
Try  699  error:  0.05009964835160871
Try  700  error:  0.05009964835160871
Try  701  error:  0.05009964835160871
Try  702  error:  0.05009964835160871
Try  703  error:  0.05009964835160871
Try  704  error:  0.05009964835160871
Try  705  error:  0.05009964835160871
Try  706  error:  0.05009964835160871
Try  707  error:  0.05009964835160871
Try  708  error:  0.05009964835160871
Try  709  error:  0.05009964835160871
Try  710  error:  0.05009964835160871
Try  711  error:  0.05009964835160871
Try  712  error:  0.05009964835160871
Try  713  error:  0.05009964835160871
Try  714  error:  0.05009964835160871
Try  715  error:  0.05009964835160871
Try  716  error:  0.05009964835160871
Try  717  er

...

Try  9794  error:  0.05009964835160871
Try  9795  error:  0.05009964835160871
Try  9796  error:  0.05009964835160871
Try  9797  error:  0.05009964835160871
Try  9798  error:  0.05009964835160871
Try  9799  error:  0.05009964835160871
Try  9800  error:  0.05009964835160871
Try  9801  error:  0.05009964835160871
Try  9802  error:  0.05009964835160871
Try  9803  error:  0.05009964835160871
Try  9804  error:  0.05009964835160871
Try  9805  error:  0.05009964835160871
Try  9806  error:  0.05009964835160871
Try  9807  error:  0.05009964835160871
Try  9808  error:  0.05009964835160871
Try  9809  error:  0.05009964835160871
Try  9810  error:  0.05009964835160871
Try  9811  error:  0.05009964835160871
Try  9812  error:  0.05009964835160871
Try  9813  error:  0.05009964835160871
Try  9814  error:  0.05009964835160871
Try  9815  error:  0.05009964835160871
Try  9816  error:  0.05009964835160871
Try  9817  error:  0.05009964835160871
Try  9818  error:  0.05009964835160871
Try  9819  error:  0.0500

Try  10005  error:  0.05009964835160871
Try  10006  error:  0.05009964835160871
Try  10007  error:  0.05009964835160871
Try  10008  error:  0.05009964835160871
Try  10009  error:  0.05009964835160871
Try  10010  error:  0.05009964835160871
Try  10011  error:  0.05009964835160871
Try  10012  error:  0.05009964835160871
Try  10013  error:  0.05009964835160871
Try  10014  error:  0.05009964835160871
Try  10015  error:  0.05009964835160871
Try  10016  error:  0.05009964835160871
Try  10017  error:  0.05009964835160871
Try  10018  error:  0.05009964835160871
Try  10019  error:  0.05009964835160871
Try  10020  error:  0.05009964835160871
Try  10021  error:  0.05009964835160871
Try  10022  error:  0.05009964835160871
Try  10023  error:  0.05009964835160871
Try  10024  error:  0.05009964835160871
Try  10025  error:  0.05009964835160871
Try  10026  error:  0.05009964835160871
Try  10027  error:  0.05009964835160871
Try  10028  error:  0.05009964835160871
Try  10029  error:  0.05009964835160871


Try  10213  error:  0.05009964835160871
Try  10214  error:  0.05009964835160871
Try  10215  error:  0.05009964835160871
Try  10216  error:  0.05009964835160871
Try  10217  error:  0.05009964835160871
Try  10218  error:  0.05009964835160871
Try  10219  error:  0.05009964835160871
Try  10220  error:  0.05009964835160871
Try  10221  error:  0.05009964835160871
Try  10222  error:  0.05009964835160871
Try  10223  error:  0.05009964835160871
Try  10224  error:  0.05009964835160871
Try  10225  error:  0.05009964835160871
Try  10226  error:  0.05009964835160871
Try  10227  error:  0.05009964835160871
Try  10228  error:  0.05009964835160871
Try  10229  error:  0.05009964835160871
Try  10230  error:  0.05009964835160871
Try  10231  error:  0.05009964835160871
Try  10232  error:  0.05009964835160871
Try  10233  error:  0.05009964835160871
Try  10234  error:  0.05009964835160871
Try  10235  error:  0.05009964835160871
Try  10236  error:  0.05009964835160871
Try  10237  error:  0.05009964835160871


Try  10418  error:  0.05009964835160871
Try  10419  error:  0.05009964835160871
Try  10420  error:  0.05009964835160871
Try  10421  error:  0.05009964835160871
Try  10422  error:  0.05009964835160871
Try  10423  error:  0.05009964835160871
Try  10424  error:  0.05009964835160871
Try  10425  error:  0.05009964835160871
Try  10426  error:  0.05009964835160871
Try  10427  error:  0.05009964835160871
Try  10428  error:  0.05009964835160871
Try  10429  error:  0.05009964835160871
Try  10430  error:  0.05009964835160871
Try  10431  error:  0.05009964835160871
Try  10432  error:  0.05009964835160871
Try  10433  error:  0.05009964835160871
Try  10434  error:  0.05009964835160871
Try  10435  error:  0.05009964835160871
Try  10436  error:  0.05009964835160871
Try  10437  error:  0.05009964835160871
Try  10438  error:  0.05009964835160871
Try  10439  error:  0.05009964835160871
Try  10440  error:  0.05009964835160871
Try  10441  error:  0.05009964835160871
Try  10442  error:  0.05009964835160871


Try  10623  error:  0.05009964835160871
Try  10624  error:  0.05009964835160871
Try  10625  error:  0.05009964835160871
Try  10626  error:  0.05009964835160871
Try  10627  error:  0.05009964835160871
Try  10628  error:  0.05009964835160871
Try  10629  error:  0.05009964835160871
Try  10630  error:  0.05009964835160871
Try  10631  error:  0.05009964835160871
Try  10632  error:  0.05009964835160871
Try  10633  error:  0.05009964835160871
Try  10634  error:  0.05009964835160871
Try  10635  error:  0.05009964835160871
Try  10636  error:  0.05009964835160871
Try  10637  error:  0.05009964835160871
Try  10638  error:  0.05009964835160871
Try  10639  error:  0.05009964835160871
Try  10640  error:  0.05009964835160871
Try  10641  error:  0.05009964835160871
Try  10642  error:  0.05009964835160871
Try  10643  error:  0.05009964835160871
Try  10644  error:  0.05009964835160871
Try  10645  error:  0.05009964835160871
Try  10646  error:  0.05009964835160871
Try  10647  error:  0.05009964835160871


In [3]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Original_parameters_result_tcp_euc_rishabh_offset_59.xlsx")

NameError: name 'env' is not defined